In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Part 1. Data Cleaning and Preparation

In [70]:
supplier_data_1 = pd.read_excel("supplier_data_1.xlsx")
supplier_data_2 = pd.read_excel("supplier_data_2.xlsx")

In [71]:
def clean_column_names(df):
    df_copy = df
    df_copy.columns = (
        df_copy.columns.str.strip() 
        .str.lower() 
        .str.replace(r"\s+", "_", regex=True)  
        .str.replace(r"[^\w\s]", "", regex=True)  
    )
    return df_copy

In [72]:
supplier_data_1 = clean_column_names(supplier_data_1)
supplier_data_2 = clean_column_names(supplier_data_2)

In [73]:
def convert_numeric_columns(df):
    df_copy = df.copy()
    
    for col in df_copy.columns:

        if df_copy[col].astype(str).str.contains(",", na=False).any():
            df_copy[col] = df_copy[col].astype(str).str.strip()
            df_copy[col] = df_copy[col].replace("nan", np.nan)
            df_copy[col] = df_copy[col].str.replace(",", "", regex=True) 
           
            mask = df_copy[col].notna()
            if df_copy.loc[mask, col].str.replace(".", "", regex=True).str.isdigit().all():
                df_copy[col] = pd.to_numeric(df_copy[col], errors="coerce")
    
    return df_copy


In [74]:

supplier_data_1 = convert_numeric_columns(supplier_data_1)

supplier_data_2 = convert_numeric_columns(supplier_data_2)

In [75]:
def standardize_datetime(df, column_name):
    def parse_date(value):
        value = str(value).strip().replace("/", "-")
        
        if " " in value:
            date_part, time_part = value.split(" ", 1)
        else:
            date_part, time_part = value, "00:00"  # Default time if missing
        
        parts = date_part.split("-")
        if len(parts) == 3:
            if len(parts[0]) == 4:  # YYYY-MM-DD
                formatted_date = f"{parts[0]}-{parts[1]}-{parts[2]} {time_part}"
            elif len(parts[0]) == 2:  # DD-MM-YYYY (swap to YYYY-MM-DD)
                formatted_date = f"{parts[2]}-{parts[1]}-{parts[0]} {time_part}"
            else:
                return pd.NaT
            
            return pd.to_datetime(formatted_date, format="%Y-%m-%d %H:%M", errors="coerce")
        else:
            return pd.NaT  
    
    df[column_name] = df[column_name].apply(parse_date)
    df[column_name] = df[column_name].dt.strftime("%Y-%m-%d %H:%M").fillna("Unknown")
    return df


In [76]:
import re

def extract_material_name(defect_note):
    
    defect_note = str(defect_note)
    if defect_note.startswith("DEKL-"):
        defect_note = defect_note[5:]
    
    #delete Z, /, +, whitespace, _ or -
    match = re.match(r"^([^Z/+\s_-]+)", defect_note)
    return match.group(1) if match else ""





In [77]:
supplier_data_2['possible_material_name'] = supplier_data_2['defect_notes'].apply(extract_material_name)
mask = supplier_data_2['material_name'].isna() & supplier_data_2['possible_material_name'].notna()
supplier_data_2.loc[mask, 'material_name'] = supplier_data_2.loc[mask, 'possible_material_name']

In [78]:
def product_type_clean(df, column_name):
    df[column_name] = df[column_name].astype(str)
    df[column_name] = df[column_name].apply(
        lambda x: 'COILS_STRIP' if x.startswith('C') 
                  else ('SHEET' if x.startswith('S') else np.nan)
    )
    
    return df


In [79]:
supplier_data_2 = product_type_clean(supplier_data_2, 'product_type')

In [80]:
supplier_data_2

,product_type,order_id,site,material_name,material_number,material_quality_norm,surface_coating,defect_notes,nominal_thickness_mm,width_mm,...,mass_min_kg,number_of_coils,delivery_earliest,delivery_latest,inco_term,buy_now_eur_per_ton,minmax_bid_eur_per_ton,co2_per_ton_max_kg,valid_until,possible_material_name
0,SHEET,436765,1 company gmbh,S235JR,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,2091,NaN,NaN,NaN,FCA,600.0,NaN,NaN,20/02/2025 11:00,S235JR
1,SHEET,436754,1 company gmbh,S355MC,1.0976,DIN EN 10149,NaN,NaN,8.057,1011,...,2411,NaN,NaN,NaN,FCA,600.0,NaN,NaN,20/02/2025 11:00,nan
2,SHEET,436755,1 company gmbh,S355MC,1.0976,DIN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,2251,NaN,NaN,NaN,FKA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
3,SHEET,436757,1 company gmbh,S355MC,1.0976,NaN,NaN,DEKL-S355MC / D2A WEH,8.057,1011,...,2401,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
4,SHEET,436758,1 company gmbh,S355MC,1.0976,DN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,2401,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,COILS_STRIP,436737,1 company gmbh,DC06,1.0873,NaN,NaN,DC06 / D2A ZUB,0.800,1385,...,4580,NaN,NaN,NaN,FCA,600.0,570.0,NaN,2025-02-20 11:00,DC06
132,SHEET,436765,1 company gmbh,S235JR,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,2091,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S235JR
133,COILS_STRIP,436283,1 company gmbh,CR180BH,NaN,VDA 239-100,NaN,CR180BHZM40/40-E ZM 90 MC OL / D2A VMB,0.712,1432,...,3160,NaN,NaN,NaN,FCA,NaN,490.0,NaN,2025-02-20 16:00,CR180BH
134,SHEET,436626,1 company gmbh,nan,1.0045,DIN EN 10025,NaN,NaN,12.008,1507,...,3541,NaN,NaN,NaN,FCA,640.0,610.0,NaN,2025-02-20 11:00,nan


In [81]:
material_name_data = dict(zip(supplier_data_2['material_name'],supplier_data_2['material_number']))


In [82]:
material_name_data

{'S235JR': 1.0038,
 'S355MC': 1.0976,
 'S355JR': 1.0045,
 'DC06': 1.0873,
 'DC01': 1.033,
 'S460MC': 1.0982,
 'S23SJR': 1.0038,
 'nan': 1.0045,
 'S35SJR': 1.0045,
 'HCT780X': 1.0943,
 'S500MC': 1.0984,
 'C45E': 1.1191,
 'E335': 1.006,
 'S420MC': 1.098,
 'DD11': 1.0332,
 'C45': 1.0503,
 'CR380LA': nan,
 'CR180BH': nan,
 'DX54D': 1.0306,
 'CR3': nan,
 'CR4': nan,
 'DX52D': 1.035,
 'CR210BH': nan,
 'HX180BD': 1.0914,
 'CR420LA': nan,
 'HX220BD': 1.0919,
 'DX51D': 1.0226,
 'CR210BHE': nan,
 'CR3GA45': nan,
 'CR270BH': nan,
 'CR5GI50': nan,
 'CR380LAGI50': nan,
 'CR5': nan,
 'CR240LAGI40': nan,
 'DX53D': 1.0355,
 'CR210BHGI75': nan,
 'DIVERSE': nan,
 'HC180B': 1.0395}

In [83]:
import math

material_name_data = {k: v for k, v in material_name_data.items() if k != "DIVERSE"}

In [84]:
filtered_data = {
    k: v 
    for k, v in material_name_data.items() 
    if pd.notna(k) and pd.notna(v)
}

reversed_data = {v: k for k, v in filtered_data.items()}

final_material_name_data = {k: v for v, k in reversed_data.items()}

final_material_name_data = {str(k): str(v) for k, v in final_material_name_data.items()}

final_material_name_data


{'S23SJR': '1.0038',
 'S355MC': '1.0976',
 'S35SJR': '1.0045',
 'DC06': '1.0873',
 'DC01': '1.033',
 'S460MC': '1.0982',
 'HCT780X': '1.0943',
 'S500MC': '1.0984',
 'C45E': '1.1191',
 'E335': '1.006',
 'S420MC': '1.098',
 'DD11': '1.0332',
 'C45': '1.0503',
 'DX54D': '1.0306',
 'DX52D': '1.035',
 'HX180BD': '1.0914',
 'HX220BD': '1.0919',
 'DX51D': '1.0226',
 'DX53D': '1.0355',
 'HC180B': '1.0395'}

In [85]:
supplier_data_2['material_name']= pd.to_numeric(supplier_data_2['material_name'], errors='coerce')



In [86]:
supplier_data_2

,product_type,order_id,site,material_name,material_number,material_quality_norm,surface_coating,defect_notes,nominal_thickness_mm,width_mm,...,mass_min_kg,number_of_coils,delivery_earliest,delivery_latest,inco_term,buy_now_eur_per_ton,minmax_bid_eur_per_ton,co2_per_ton_max_kg,valid_until,possible_material_name
0,SHEET,436765,1 company gmbh,NaN,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,2091,NaN,NaN,NaN,FCA,600.0,NaN,NaN,20/02/2025 11:00,S235JR
1,SHEET,436754,1 company gmbh,NaN,1.0976,DIN EN 10149,NaN,NaN,8.057,1011,...,2411,NaN,NaN,NaN,FCA,600.0,NaN,NaN,20/02/2025 11:00,nan
2,SHEET,436755,1 company gmbh,NaN,1.0976,DIN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,2251,NaN,NaN,NaN,FKA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
3,SHEET,436757,1 company gmbh,NaN,1.0976,NaN,NaN,DEKL-S355MC / D2A WEH,8.057,1011,...,2401,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
4,SHEET,436758,1 company gmbh,NaN,1.0976,DN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,2401,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S355MC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,COILS_STRIP,436737,1 company gmbh,NaN,1.0873,NaN,NaN,DC06 / D2A ZUB,0.800,1385,...,4580,NaN,NaN,NaN,FCA,600.0,570.0,NaN,2025-02-20 11:00,DC06
132,SHEET,436765,1 company gmbh,NaN,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,2091,NaN,NaN,NaN,FCA,600.0,NaN,NaN,2025-02-20 11:00,S235JR
133,COILS_STRIP,436283,1 company gmbh,NaN,NaN,VDA 239-100,NaN,CR180BHZM40/40-E ZM 90 MC OL / D2A VMB,0.712,1432,...,3160,NaN,NaN,NaN,FCA,NaN,490.0,NaN,2025-02-20 16:00,CR180BH
134,SHEET,436626,1 company gmbh,NaN,1.0045,DIN EN 10025,NaN,NaN,12.008,1507,...,3541,NaN,NaN,NaN,FCA,640.0,610.0,NaN,2025-02-20 11:00,nan


In [87]:
import pandas as pd
from math import nan

class BidirectionalDict(dict):
    """
    A dictionary that allows bidirectional lookup (key → value and value → key).
    It normalizes keys and values to stripped strings, and handles `'nan'` as a special case.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._normalize()
        self._update_inverse()

    def _normalize(self):
        normalized = {
            str(k).strip(): str(v).strip() if v not in [None, 'nan'] else None
            for k, v in self.items()
        }
        self.clear()
        super().update(normalized)

    def _update_inverse(self):
        self.inverse = {
            v: k for k, v in self.items() if v not in [None, 'nan']
        }

    def __setitem__(self, key, value):
        key = str(key).strip()
        value = str(value).strip() if value not in [None, 'nan'] else None
        super().__setitem__(key, value)
        self._update_inverse()  

    def update(self, *args, **kwargs):
        super().update(*args, **kwargs)
        self._normalize()
        self._update_inverse()  


def apply_material_mapping(df, mapping):
    """
    Applies bidirectional material mapping to a dataset.
    
    For rows where one of the columns (material_name or material_number) is missing,
    fill the missing value using the bidirectional mapping.
    
    Parameters:
        df (DataFrame): Dataset containing "material_name" and "material_number".
        mapping (BidirectionalDict): Mapping for bidirectional lookup.
    
    Returns:
        DataFrame: Updated dataset with missing values filled.
    """

    df_copy = df.copy()

    for col in ["material_name", "material_number"]:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].apply(lambda x: str(x).strip() if pd.notna(x) else x)

    def fill_row(row):
        mat_name = row.get("material_name")
        mat_number = row.get("material_number")
        
        if (mat_name is None or pd.isna(mat_name)) and pd.notna(mat_number):
            key = str(mat_number).strip() if pd.notna(mat_number) else None
            row["material_name"] = mapping.inverse.get(key, mat_name)
        
        if (mat_number is None or pd.isna(mat_number)) and pd.notna(mat_name):
            key = str(mat_name).strip() if pd.notna(mat_name) else None
            row["material_number"] = mapping.get(key, mat_number)
        
        return row

    df_copy = df_copy.apply(fill_row, axis=1)
    return df_copy


In [88]:
material_mapping = BidirectionalDict(final_material_name_data)
supplier_data_2= apply_material_mapping(supplier_data_2, material_mapping)


In [89]:
supplier_data_2['possible_material_name'] = supplier_data_2['defect_notes'].apply(extract_material_name)
mask = supplier_data_2['material_name'].isna() & supplier_data_2['possible_material_name'].notna()
supplier_data_2.loc[mask, 'material_name'] = supplier_data_2.loc[mask, 'possible_material_name']


Part 2. Preparation for tables merge

In [90]:
supplier_data_2['breite'] = supplier_data_2['width_mm'] 

In [91]:
# Checking for a potential hidden key
hidden_key_candidates = set(supplier_data_1.columns).intersection(set(supplier_data_2.columns))

table1_unique_keys = supplier_data_1.nunique()
table2_unique_keys = supplier_data_2.nunique()

hidden_key_candidates, table1_unique_keys, table2_unique_keys


({'breite'},
 werksgüte                              32
 bestellgütentext                       28
 nenndicke_nnnnn_mm_mit_dezimalpunkt    50
 breite                                 71
 länge                                  82
 gewicht_kg                             78
 cluster                                 6
 sigehalt                               34
 mngehalt                               33
 pgehalt                                21
 sgehalt                                17
 crgehalt                               33
 nigehalt                               19
 mogehalt                               26
 vgehalt                                15
 cugehalt                               29
 nbgehalt                               28
 tigehalt                               33
 algehalt                               27
 bgehalt                                11
 streckgrenze                           14
 zugfestigkeit                          15
 dehnung                                1

In [92]:
hidden_key_candidates 
#let's merge on 'breite'


# IMPORTANT: By looking in 2 datasets I decided that it tables should be joined LEFT.
# REASONS: it's more important to have product_type and material_name, material_namber 
# than werkgute and bestellgutetext 

{'breite'}

In [93]:
supplier_data_1['breite'] = supplier_data_1['breite'].astype(str).str.strip()
supplier_data_2['breite'] = supplier_data_2['breite'].astype(str).str.strip()

supplier_data_1['breite'] = pd.to_numeric(supplier_data_1['breite'], errors='coerce').astype('float64')
supplier_data_2['breite'] = pd.to_numeric(supplier_data_2['breite'], errors='coerce').astype('float64')


In [94]:
inventory_data = supplier_data_2.merge(supplier_data_1, how = 'left', on = 'breite')

In [95]:
inventory_data

,product_type,order_id,site,material_name,material_number,material_quality_norm,surface_coating,defect_notes,nominal_thickness_mm,width_mm,...,mogehalt,vgehalt,cugehalt,nbgehalt,tigehalt,algehalt,bgehalt,streckgrenze,zugfestigkeit,dehnung
0,SHEET,436765,1 company gmbh,S23SJR,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SHEET,436754,1 company gmbh,S355MC,1.0976,DIN EN 10149,NaN,NaN,8.057,1011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHEET,436755,1 company gmbh,S355MC,1.0976,DIN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SHEET,436757,1 company gmbh,S355MC,1.0976,NaN,NaN,DEKL-S355MC / D2A WEH,8.057,1011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SHEET,436758,1 company gmbh,S355MC,1.0976,DN EN 10149,NaN,DEKL-S355MC / D2A WEH,8.057,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,COILS_STRIP,436737,1 company gmbh,DC06,1.0873,NaN,NaN,DC06 / D2A ZUB,0.800,1385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,SHEET,436765,1 company gmbh,S23SJR,1.0038,DIN EN 10025,NaN,DEKL-S235JR / D2A EID,11.859,1509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,COILS_STRIP,436283,1 company gmbh,CR180BH,NaN,VDA 239-100,NaN,CR180BHZM40/40-E ZM 90 MC OL / D2A VMB,0.712,1432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,SHEET,436626,1 company gmbh,S35SJR,1.0045,DIN EN 10025,NaN,NaN,12.008,1507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
def assign_product_type(df):
    mask = df['product_type'].isna()
    
    # For rows with null product_type, assign based on whether 'length_mm' is null
    df.loc[mask, 'product_type'] = np.where(
        df.loc[mask, 'length_mm'].isna(),
        'COILS_STRIP',  # when 'length_mm' is null
        'SHEET'         # when 'length_mm' is not null
    )
    return df


In [97]:
inventory_data = assign_product_type(inventory_data)

In [98]:
set(inventory_data.product_type)

{'COILS_STRIP', 'SHEET'}

In [99]:
inventory_data.to_csv("task1_inventory_data.csv")